| Stage 1: Continue Pretraining   | 增量预训练，在海量领域文本数据上继续预训练GPT模型，以注入领域知识       | [pretraining.py](https://github.com/shibing624/MedicalGPT/blob/main/pretraining.py) | [run_pt.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_pt.sh)    | [run_pretraining.ipynb](https://github.com/shibing624/MedicalGPT/blob/main/run_pretraining.ipynb)    | [Open In Colab](https://colab.research.google.com/github/shibing624/MedicalGPT/blob/main/run_pretraining.ipynb)           |

# Stage 1: Continue Pretraining

第一阶段：PT(Continue PreTraining)增量预训练，在海量领域文本数据上二次预训练LLaMA类GPT模型，以注入领域知识


#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m`
2. 数据集：PT阶段使用的是中文天龙八部小说部分文本和英文书籍部分文本，位于`data/pretrain`文件夹

## 配置运行环境

本地执行可注释以下配置环境的命令，colab执行要打开注释，用于配置环境

colab建议使用T4 GPU训练，设置方式：`代码执行程序 -> 更改运行时类型 -> 运行时类型：Python3，硬件加速器：GPU，GPU类型：T4 -> 保存`

步骤：
1. 下载最新代码到本地
2. 安装依赖包

依赖包如下，保证最新版本：

```
loguru
transformers
datasets
tensorboard
tqdm
peft
trl
```

In [32]:
!git clone --depth 1 https://github.com/shibing624/MedicalGPT.git
%cd MedicalGPT
%ls
!pip install -r requirements.txt

## 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [14]:
%ls ./data/pretrain/

en_article_tail500.txt  tianlongbabu.txt


In [ ]:
!python pretraining.py \
    --model_type bloom \
    --model_name_or_path bigscience/bloomz-560m \
    --train_file_dir ./data/pretrain \
    --validation_file_dir ./data/pretrain \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --seed 42 \
    --fp16 \
    --max_train_samples 10000 \
    --max_eval_samples 10 \
    --num_train_epochs 0.5 \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --block_size 1024 \
    --output_dir outputs-pt-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

In [28]:
%ls outputs-pt-v1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
adapter_config.json  runs/                    train_results.json
adapter_model.bin    special_tokens_map.json  trainer_state.json
all_results.json     tokenizer.json           training_args.bin
eval_results.json    tokenizer_config.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

本节完。